# Preparação do ambiente

Montar o drive para acessar os arquivos

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


Instalar as dependências e importar as libs

In [2]:
!pip install scikit-multilearn==0.2.0
!pip install scikit-learn==0.24

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [3]:
from sklearn.feature_extraction.text import TfidfVectorizer
from skmultilearn.problem_transform import ClassifierChain
from skmultilearn.problem_transform import BinaryRelevance
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.multiclass import OneVsRestClassifier
from sklearn.metrics import hamming_loss
from skmultilearn.adapt import MLkNN
import pandas as pd
import numpy as np

# Código

## Parte 1: Entendendo a classificação multilabel

In [4]:
perguntas = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/Di2Win/"+
"Alura_2023/Machine Learning para Negócios Digitais/" + 
"Classificação multilabel de textos: múltiplos contextos de NLP/" + 
"stackoverflow_perguntas.csv")

In [4]:
perguntas.sample(10)

,Perguntas,Tags
2186,"Bom, uma simples consulta ajax, php e mysql. E...",jquery
739,Dúvida: Gostaria de saber um meio de utilizar ...,jquery
3981,No momento estou utilizando o Linux Mint 18.2 ...,node.js
3349,Bom eu removo o CSS de um determinado elemento...,jquery
219,Boa noite. Tenho um js onde preciso que seja e...,jquery
3177,Tenho esse código para colocar máscara no camp...,jquery
818,Dias atrás fiz uma pergunta sobre como adicion...,html
4107,Estou tentando criar uma árvore de CheckBox po...,html angular node.js
1,"Gostaria de fazer testes unitários no Node.js,...",node.js
1849,Quero gerar thumbnails de imagens que o usuári...,html


## Parte 2: Preparando os dados

In [ ]:
print(f'Quantidade de linhas: {len(perguntas)}')

Quantidade de linhas: 5408


In [ ]:
print(f'Quantidade de Tags únicas {len(perguntas.Tags.unique())}')

Quantidade de Tags únicas 37


In [ ]:
# Lista de Tags únicas dentro do DF
perguntas.Tags.unique()

array(['node.js', 'jquery', 'html', 'html angular ', 'html ', 'angular',
       'angular ', 'jquery html  ', 'jquery ', 'jquery html',
       'jquery html ', 'html angular', 'angular node.js ', 'html  ',
       'jquery html angular', 'node.js ', 'html jquery', 'html jquery ',
       'jquery angular  ', 'html node.js', 'jquery  ', 'angular node.js',
       'jquery angular', 'html node.js ', 'jquery node.js ', 'angular  ',
       'jquery angular ', 'jquery html angular ', 'node.js html ',
       ' node.js', 'node.js html', 'html angular  ', 'jquery node.js',
       'angular html', 'html angular  node.js', 'jquery html node.js',
       'html angular node.js'], dtype=object)

Existem elementos dentro das tags que, por serem strings, estão gerando "duplicidades". 

**Ex.:** 'html' != 'html '

In [5]:
# Criar lista de tags unicas
tag_list = list()
for tag_value in perguntas.Tags.unique():
  tags = tag_value.split(' ')
  for tag in tags:
    if tag not in tag_list and tag != '':
      tag_list.append(tag)

In [6]:
tag_list

['node.js', 'jquery', 'html', 'angular']

In [7]:
def criar_nova_coluna(lista_tags, dataframe, nome_tags="Tags"):
  """
  Criar nova coluna para o dataframe
  :param lista_tags: Lista de tags únicas
  :param dataframe: Dataframe que será alterado
  :param nome_tags: Nome da coluna que contém as tags.

  """
  for tag in lista_tags:
    coluna = []
    for linha_tag in dataframe[nome_tags]:
      if tag in linha_tag:
        coluna.append(1)
      else:
        coluna.append(0)
    
    dataframe[tag] = coluna

In [8]:
criar_nova_coluna(tag_list, perguntas)

In [9]:
perguntas.sample(5)

,Perguntas,Tags,node.js,jquery,html,angular
407,Tenho a seguinte tabela: CODE Como deu para...,jquery,0,1,0,0
4057,"Boa noite, estou com uma duvida em CSS Estou ...",html,0,0,1,0
374,No back end quando faço submit ele apenas me a...,html,0,0,1,0
3765,Estou com problemas com um objeto no Node.Js c...,angular node.js,1,0,0,1
179,"Vamos supor, clico no botão Mais e vou pra fre...",html,0,0,1,0


## Parte 3: Dados de treino e teste

Como a base de treino e teste deve ter a mesma proporção ou próximo a isso
de arquivos com as mesmas saídas, se faz necessário unificar as colunas
de tags em apenas uma única coluna.

In [9]:
# Unir as colunas de tags individuais em apenas uma coluna
lista_zip_tags = list(zip(perguntas["node.js"],
                     perguntas["jquery"],
                     perguntas["html"],
                     perguntas["angular"]))

In [ ]:
print(lista_zip_tags)

[(1, 0, 0, 0), (1, 0, 0, 0), (0, 1, 0, 0), (0, 0, 1, 0), (0, 0, 1, 1), (0, 0, 1, 0), (0, 0, 0, 1), (1, 0, 0, 0), (0, 0, 0, 1), (1, 0, 0, 0), (0, 0, 0, 1), (0, 1, 0, 0), (1, 0, 0, 0), (0, 0, 0, 1), (0, 1, 1, 0), (0, 0, 1, 0), (0, 0, 0, 1), (0, 1, 0, 0), (0, 0, 1, 0), (0, 0, 0, 1), (0, 1, 0, 0), (0, 0, 1, 0), (0, 0, 1, 0), (0, 0, 0, 1), (0, 0, 1, 0), (0, 1, 1, 0), (0, 1, 0, 0), (0, 1, 1, 0), (0, 1, 1, 0), (0, 1, 0, 0), (0, 0, 0, 1), (0, 0, 1, 0), (0, 0, 1, 1), (0, 0, 1, 1), (0, 1, 0, 0), (0, 0, 1, 0), (0, 1, 0, 0), (0, 0, 1, 0), (0, 0, 1, 1), (0, 1, 0, 0), (0, 1, 0, 0), (0, 0, 1, 0), (0, 1, 0, 0), (0, 0, 1, 0), (0, 0, 1, 0), (0, 1, 0, 0), (0, 0, 0, 1), (0, 1, 0, 0), (0, 1, 1, 0), (0, 0, 1, 0), (0, 1, 1, 0), (0, 1, 0, 0), (0, 0, 0, 1), (0, 1, 1, 0), (0, 0, 1, 0), (1, 0, 0, 0), (0, 1, 1, 0), (1, 0, 0, 0), (0, 1, 0, 0), (0, 1, 1, 0), (0, 0, 0, 1), (0, 1, 0, 0), (0, 0, 1, 0), (0, 0, 1, 0), (0, 1, 1, 0), (0, 1, 0, 0), (0, 1, 0, 0), (0, 0, 1, 0), (0, 1, 0, 0), (0, 1, 1, 0), (0, 1, 0, 0), (0, 0

In [11]:
# Adicionar a nova coluna ao dataframe
perguntas["todas_tags"] = lista_zip_tags
perguntas.sample(5)

,Perguntas,Tags,node.js,jquery,html,angular,todas_tags
4176,Tenho o objeto array CODE : É aplicado um *...,angular,0,0,0,1,"(0, 0, 0, 1)"
3053,Bom dia! Estamos criando uma página super sim...,html,0,0,1,0,"(0, 0, 1, 0)"
1252,Tenho uma aplicação de um chat e estou trabalh...,node.js,1,0,0,0,"(1, 0, 0, 0)"
740,Digamos que eu tenha um formulário com os camp...,jquery html,0,1,1,0,"(0, 1, 1, 0)"
4653,"Bom dia a todos , Comecei a estudar javascri...",jquery html,0,1,1,0,"(0, 1, 1, 0)"


In [12]:
# Criando a base de teste e treino
perguntas_treino, perguntas_teste, tags_treino, tags_teste = train_test_split(
    perguntas.Perguntas, 
    perguntas.todas_tags,
    test_size=0.2,
    random_state=123
)

In [ ]:
perguntas_treino

1577       array1 = [1,2,3];   array2 = ["um","dois","...
1927    Não sei se fui claro no título, mas quem é da ...
3409    Alguém sabe me dizer qual a melhor forma de re...
4606    Estou com problemas ao tentar validar campos d...
5237    Preciso copiar um valor de dentro de um CODE  ...
                              ...                        
5218    Tenho um sisteminha, para mudar o layout da pá...
4060    Como fazer alto scoll ao carregar a página?  E...
1346    Explicação:  Tenho uma CODE  pai que contém du...
3454    Estou querendo fazer um sistema onde eu iria t...
3582    Galera eu to com um problemão, ja pesquisei ba...
Name: Perguntas, Length: 4326, dtype: object

Como é possível notar, os dados de entrada está sendo texto,
mas computadores lidam melhor com dados numéricos. 
Para auxiliar o modelo a ser desenvolvido, o texto será vetorizado
utilizando a técnica de TF-IDF.

In [13]:
vetorizar = TfidfVectorizer(max_features=5000,
                            max_df=0.85)

In [14]:
vetorizar.fit(perguntas.Perguntas)

TfidfVectorizer(max_df=0.85, max_features=5000)

In [15]:
perguntas_treino_tfidf = vetorizar.transform(perguntas_treino)
perguntas_teste_tfidf = vetorizar.transform(perguntas_teste)

In [16]:
print(f'Shape da base de treino: {perguntas_treino_tfidf.shape}')
print(f'Shape da base de treino: {perguntas_teste_tfidf.shape}')

Shape da base de treino: (4326, 5000)
Shape da base de treino: (1082, 5000)


## Parte 4: Criando o primeiro modelo

Aplicando um classificador de relevância binária, ou seja, 
para cada uma das colunas será criado um modelo.

In [17]:
regressao_logistica = LogisticRegression()
classificador_onevsrest = OneVsRestClassifier(estimator=regressao_logistica)

É necessário converter a base de treino e teste para o formato array

In [19]:
tags_treino_array = np.asarray(list(tags_treino))
tags_teste_array = np.asarray(list(tags_teste))

In [25]:
print(tags_treino_array)

[[0 1 0 0]
 [0 1 0 0]
 [0 0 1 0]
 ...
 [0 1 1 0]
 [0 0 1 0]
 [0 1 1 0]]


In [21]:
classificador_onevsrest.fit(perguntas_treino_tfidf, tags_treino_array)

OneVsRestClassifier(estimator=LogisticRegression())

In [22]:
resultado_onevsrest = classificador_onevsrest.score(perguntas_teste_tfidf,
                                                    tags_teste_array)
print("Resultado: {0: .2f}%".format(resultado_onevsrest*100))

Resultado:  41.68%


## Parte 5: Métricas de avaliação

O cálculo da acurácia feita anteriormente é uma predição do valor em conjunto,
ou seja, apenas se todas as 4 colunas forem preditas corretamente será
contabilizado um acerto.

Para melhorar a avaliação usaremos a *Hamming Loss*.

In [23]:
previsao_onevsrest = classificador_onevsrest.predict(perguntas_teste_tfidf)
hamming_loss_onevsrest = hamming_loss(tags_teste_array,
                                      previsao_onevsrest)

In [35]:
print("Hamming Loss: {0: .2f}".format(hamming_loss_onevsrest))

Hamming Loss:  0.19


## Parte 6: Captando relação com classificação em cadeia

Calculando a correlação das colunas:

In [37]:
perguntas.corr()

,node.js,jquery,html,angular
node.js,1.000000,-0.321485,-0.273523,-0.101787
jquery,-0.321485,1.000000,-0.253977,-0.366269
html,-0.273523,-0.253977,1.000000,-0.286706
angular,-0.101787,-0.366269,-0.286706,1.000000


O novo modelo que irá ser treinado utilizará as colunas anteriores como entradas
para realizar a predição da subsequente. 

Por exemplo, o modelo 1 irá predizer a coluna **node.js**, o modelo 2 irá usar
a informação da coluna node.js para predizer a coluna **jquery** e assim segue.

In [24]:
classificador_cadeia = ClassifierChain(regressao_logistica)

In [25]:
classificador_cadeia.fit(perguntas_treino_tfidf, tags_treino_array)
resultado_cadeia = classificador_cadeia.score(perguntas_teste_tfidf, 
                                              tags_teste_array)
print("Resultado: {0: .2f}%".format(resultado_cadeia*100))

Resultado:  49.82%


In [26]:
previsao_cadeia = classificador_cadeia.predict(perguntas_teste_tfidf)
hamming_loss_cadeia = hamming_loss(tags_teste_array, 
                                   previsao_cadeia)
print("Hamming Loss: {0: .2f}".format(hamming_loss_cadeia))

Hamming Loss:  0.21


### Bônus: Usando o skmultilearn para o problema de Relevância Binária ⭐

In [27]:
classificador_br = BinaryRelevance(regressao_logistica)
classificador_br.fit(perguntas_treino_tfidf, tags_treino_array)
resultado_br = classificador_br.score(perguntas_teste_tfidf, 
                                      tags_teste_array)
print("Resultado: {0: .2f}%".format(resultado_br*100))


previsao_br = classificador_br.predict(perguntas_teste_tfidf)
hamming_loss_br = hamming_loss(tags_teste_array, 
                                   previsao_br)
print("Hamming Loss: {0: .2f}".format(hamming_loss_br))

Resultado:  41.68%
Hamming Loss:  0.19


Mesmo resultado obtido com duas *libs* diferente, o que era esperado. 😴

## Parte 7: Adaptando algoritmos para classificação

Também podemos tratar o problema como um agrupamento, utilizando o algoritmo de ML-KNN, no qual a partir de distâncias uma determinada saída será classificada baseado no seus vizinho, podendo ter mais de uma classe.

In [18]:
classificador_mlknn = MLkNN(k=10)

In [20]:
classificador_mlknn.fit(perguntas_treino_tfidf, tags_treino_array)
resultado_mlknn = classificador_mlknn.score(perguntas_teste_tfidf, 
                                            tags_teste_array)
print("Resultado: {0: .2f}%".format(resultado_mlknn*100))


previsao_mlknn = classificador_mlknn.predict(perguntas_teste_tfidf)
hamming_loss_mlknn = hamming_loss(tags_teste_array, 
                                  previsao_mlknn)
print("Hamming Loss: {0: .2f}".format(hamming_loss_mlknn))

/usr/local/lib/python3.9/dist-packages/sklearn/utils/validation.py:70: FutureWarning: Pass n_neighbors=10 as keyword args. From version 1.0 (renaming of 0.25) passing these as positional arguments will result in an error
  warnings.warn(f"Pass {args_msg} as keyword args. From version "


Resultado:  32.53%
Hamming Loss:  0.25


Verificar as classificações feitas:

In [30]:
resultados_classificacao = pd.DataFrame()
resultados_classificacao["perguntas"] = perguntas_teste.values
resultados_classificacao["tags real"] = list(tags_teste)
resultados_classificacao["BR"] = list(previsao_br.toarray())
resultados_classificacao["cadeia"] = list(previsao_cadeia.toarray())
resultados_classificacao["MLkNN"] = list(previsao_mlknn.toarray())

In [31]:
resultados_classificacao.sample(5)

,perguntas,tags real,BR,cadeia,MLkNN
208,"É porque o site teria uma Web Rádio, e eu não ...","(0, 1, 0, 0)","[0, 1, 1, 0]","[0.0, 1.0, 1.0, 0.0]","[0, 0, 1, 0]"
59,Estou desenvolvendo um aplicativo utilizando I...,"(0, 0, 0, 1)","[0, 0, 0, 0]","[0.0, 0.0, 0.0, 1.0]","[0, 0, 0, 1]"
431,Eu possuo várias estruturas que seguem um esti...,"(0, 1, 0, 0)","[0, 1, 0, 0]","[0.0, 1.0, 0.0, 0.0]","[0, 0, 1, 0]"
191,Tenho uma controller: CODE E no meu html: ...,"(0, 0, 0, 1)","[0, 0, 0, 1]","[0.0, 0.0, 0.0, 1.0]","[0, 0, 0, 1]"
687,Tenho o seguinte código CODE Preciso envia...,"(1, 0, 0, 0)","[0, 0, 0, 0]","[0.0, 1.0, 0.0, 0.0]","[0, 0, 1, 0]"
